## Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
updated_applications = application_df.drop(columns=['NAME' , 'EIN'], axis=1)

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = updated_applications['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
reduced_applications = updated_applications['APPLICATION_TYPE'].value_counts()[updated_applications['APPLICATION_TYPE'].value_counts()>10]
reduced_applications

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = updated_applications['APPLICATION_TYPE'].value_counts(ascending=False).index[8:].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count[classification_count>1]
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts(ascending=False).index[4:].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df, dtype=float)

In [13]:
# Split our preprocessed data into our features and target arrays
x = numeric_data.drop(columns = ['IS_SUCCESSFUL']).values
y = numeric_data['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(x,y, random_state=42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
print("Input Features =", number_input_features)
hidden_nodes_layer1 = 10
print("Hidden Nodes - Layer 1 =", hidden_nodes_layer1)
hidden_nodes_layer2 = 10
print("Hidden Nodes - Layer 2 =", hidden_nodes_layer2)
hidden_nodes_layer3 = 15
print("Hidden Nodes - Layer 3 =", hidden_nodes_layer3)

#Keras Sequential Model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Input Features = 19611
Hidden Nodes - Layer 1 = 10
Hidden Nodes - Layer 2 = 10
Hidden Nodes - Layer 3 = 15
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                196120    
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 15)                165       
                                                                 
 dense_7 (Dense)             (None, 1)                 16        
                                                                 
Total params: 196,411
Trainable params: 196,411
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.4942 - accuracy: 0.7651
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.3181 - accuracy: 0.8473
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1854 - accuracy: 0.9249
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1182 - accuracy: 0.9547
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0990 - accuracy: 0.9612
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0947 - accuracy: 0.9617
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0931 - accuracy: 0.9621
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0923 - accuracy: 0.9621
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0921 - accuracy: 0.9625
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.0916 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7802 - accuracy: 0.6241 - 705ms/epoch - 3ms/step
Loss: 0.7801631689071655, Accuracy: 0.6241399645805359


In [20]:
# Export our model to HDF5 file
nn.save('../AlphabetSoupCharity_Optimization1.h5')